In [10]:
import pyodbc

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=.\SQLEXPRESS;'
    'DATABASE=master;'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()
cursor.execute("SELECT 1")
print(cursor.fetchone())


(1,)


<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Rasulbek907\AppData\Local\Temp\ipykernel_6992\803325866.py:5: SyntaxWarning: invalid escape sequence '\S'
  'SERVER=.\SQLEXPRESS;'


In [15]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine

# 1️⃣ CSV o'qish
csv_file = r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Preprosessed\Missing_Value.csv"
df = pd.read_csv(csv_file)
print(f"✅ CSV fayl o'qildi: {len(df)} yozuv")

# 2️⃣ SQL Server ulanishi (Windows Authentication)
server = r'.\SQLEXPRESS'
database = 'Keggle_Database'
driver = 'ODBC Driver 17 for SQL Server'

# master bazaga ulanish (database yaratish uchun)
conn_str_master = rf'DRIVER={{{driver}}};SERVER={server};DATABASE=master;Trusted_Connection=yes;'
conn = pyodbc.connect(conn_str_master, autocommit=True)
cursor = conn.cursor()
cursor.execute(f"IF DB_ID('{database}') IS NULL CREATE DATABASE [{database}]")
cursor.close()
conn.close()
print(f"✅ Database '{database}' tekshirildi / yaratildi")

# 3️⃣ SQLAlchemy engine yaratish
engine = create_engine(
    f"mssql+pyodbc://{server}/{database}?driver={driver}&trusted_connection=yes",
    echo=True
)

# 4️⃣ CSV import qilish
table_name = 'Keggle_Dataset'
try:
    with engine.begin() as conn:
        df.to_sql(table_name, con=conn, if_exists='replace', index=False, schema='dbo')
    print(f"✅ CSV muvaffaqiyatli '{table_name}' jadvaliga yuklandi!")
except Exception as e:
    print(f"❌ Xato yuz berdi: {e}")


✅ CSV fayl o'qildi: 4209 yozuv
✅ Database 'Keggle_Database' tekshirildi / yaratildi
2025-11-11 11:35:46,773 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2025-11-11 11:35:46,774 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-11 11:35:46,776 INFO sqlalchemy.engine.Engine SELECT schema_name()
2025-11-11 11:35:46,777 INFO sqlalchemy.engine.Engine [generated in 0.00069s] ()
2025-11-11 11:35:46,807 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2025-11-11 11:35:46,809 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ()
2025-11-11 11:35:46,811 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2025-11-11 11:35:46,814 INFO sqlalchemy.engine.Engine [generated in 0.00323s] ()
2025-11-11 11:35:46,899 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-11 11:35:46,906 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].

In [21]:
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px

# -----------------------------
# 1️⃣ SQL Server ulanish
# -----------------------------
server = r'.\SQLEXPRESS'
database = 'Keggle_Database'
driver = 'ODBC Driver 17 for SQL Server'

engine = create_engine(
    f"mssql+pyodbc://{server}/{database}?driver={driver}&trusted_connection=yes"
)

# -----------------------------
# 2️⃣ Eng yangi 10 datasetni olish
# -----------------------------
query = """
SELECT TOP 10 *
FROM dbo.Keggle_Dataset
ORDER BY lastupdated DESC
"""
df_latest = pd.read_sql(query, con=engine)
print("✅ Eng yangi 10 dataset o'qildi:")
print(df_latest)

# -----------------------------
# 3️⃣ Numeric ustunlarni aniqlash
# -----------------------------
numeric_cols = df_latest.select_dtypes(include='number').columns.tolist()
print(f"Numeric ustunlar: {numeric_cols}")

# -----------------------------
# 4️⃣ Scatter plot (birinchi 2 numeric ustun)
# -----------------------------
if len(numeric_cols) >= 2:
    fig_scatter = px.scatter(df_latest,
                             x=numeric_cols[0],
                             y=numeric_cols[1],
                             size=numeric_cols[0],  # marker o'lchami
                             color=df_latest.select_dtypes(include='object').columns[0] if len(df_latest.select_dtypes(include='object').columns) > 0 else None,
                             hover_data=df_latest.columns,
                             title=f"Scatter plot: {numeric_cols[0]} vs {numeric_cols[1]}")
    fig_scatter.show()

# -----------------------------
# 5️⃣ Histogram barcha numeric ustunlar uchun
# -----------------------------
for col in numeric_cols:
    fig_hist = px.histogram(df_latest,
                            x=col,
                            nbins=10,
                            title=f"Histogram: {col}")
    fig_hist.show()

# -----------------------------
# 6️⃣ Bar chart categorical ustunlar uchun
# -----------------------------
categorical_cols = df_latest.select_dtypes(include='object').columns.tolist()
for cat_col in categorical_cols:
    # Numeric ustunlardan birini bar chart uchun ishlatamiz (masalan birinchi)
    if numeric_cols:
        fig_bar = px.bar(df_latest,
                         x=cat_col,
                         y=numeric_cols[0],
                         color=cat_col,
                         title=f"Bar chart: {numeric_cols[0]} by {cat_col}")
        fig_bar.show()


✅ Eng yangi 10 dataset o'qildi:
   Unnamed: 0.1  Unnamed: 0       id  \
0          2214        2214  8280018   
1          3578        3578  2448797   
2          2506        2506  3010373   
3          1261        1261  8360275   
4           472         472  8275485   
5           101         101  8294796   
6          2114        2114  8126455   
7          3680        3680  1333356   
8           197         197     1346   
9          3535        3535  8127986   

                                                 ref  \
0        mayankkumarpoddar/smartprix-laptops-dataset   
1         sandy1811/data-science-interview-questions   
2        svaningelgem/crypto-currencies-daily-prices   
3                barefootjoey/powerball-draw-history   
4               teocalvo/teomewhy-education-platform   
5  isaaclopgu/cryptocurrency-historical-prices-to...   
6              viniciusqroz/ethereum-historical-data   
7  code1110/yfinance-stock-price-data-for-numerai...   
8                mcziel

In [25]:
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px
from dash import Dash, dcc, html

# -----------------------------
# 1️⃣ SQL Server ulanish va eng yangi 10 dataset
# -----------------------------
server = r'.\SQLEXPRESS'
database = 'Keggle_Database'
driver = 'ODBC Driver 17 for SQL Server'

engine = create_engine(
    f"mssql+pyodbc://{server}/{database}?driver={driver}&trusted_connection=yes"
)

query = """
SELECT TOP 10 *
FROM dbo.Keggle_Dataset
ORDER BY lastupdated DESC
"""
df_latest = pd.read_sql(query, con=engine)

# -----------------------------
# 2️⃣ Numeric va categorical ustunlarni aniqlash
# -----------------------------
numeric_cols = df_latest.select_dtypes(include='number').columns.tolist()
categorical_cols = df_latest.select_dtypes(include='object').columns.tolist()

# -----------------------------
# 3️⃣ Dash app yaratish
# -----------------------------
app = Dash(__name__)

# Scatter plot (birinchi 2 numeric ustun)
if len(numeric_cols) >= 2:
    scatter_fig = px.scatter(df_latest,
                             x=numeric_cols[0],
                             y=numeric_cols[1],
                             size=numeric_cols[0],
                             color=categorical_cols[0] if categorical_cols else None,
                             hover_data=df_latest.columns,
                             title=f"Scatter plot: {numeric_cols[0]} vs {numeric_cols[1]}")
else:
    scatter_fig = None

# Histogramlar barcha numeric ustunlar uchun
hist_figs = [px.histogram(df_latest, x=col, nbins=10, title=f"Histogram: {col}") for col in numeric_cols]

# Bar chartlar categorical ustunlar uchun (birinchi numeric ustun bilan)
bar_figs = [px.bar(df_latest, x=cat, y=numeric_cols[0], color=cat, title=f"Bar chart: {numeric_cols[0]} by {cat}") 
            for cat in categorical_cols if numeric_cols]

# -----------------------------
# 4️⃣ Layout tayyorlash
# -----------------------------
graphs = []

if scatter_fig:
    graphs.append(dcc.Graph(figure=scatter_fig))

for fig in hist_figs:
    graphs.append(dcc.Graph(figure=fig))

for fig in bar_figs:
    graphs.append(dcc.Graph(figure=fig))

app.layout = html.Div(children=graphs, style={'columnCount': 2})  # 2 ustunli grid

# -----------------------------
# 5️⃣ Appni ishga tushirish
# -----------------------------
if __name__ == "__main__":
    app.run(debug=True)  # eski app.run_server() eskirgan